In [14]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import os
import tensorflow as tf

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 4s 0us/step


In [3]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

In [4]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5
938/938 [==============================] - 19s 18ms/step - loss: 0.1797 - accuracy: 0.9455
Epoch 2/5
938/938 [==============================] - 16s 17ms/step - loss: 0.0506 - accuracy: 0.9843
Epoch 3/5
938/938 [==============================] - 16s 17ms/step - loss: 0.0361 - accuracy: 0.9887
Epoch 4/5
938/938 [==============================] - 15s 16ms/step - loss: 0.0277 - accuracy: 0.9916
Epoch 5/5
313/313 [==============================] - 4s 10ms/step - loss: 0.0292 - accuracy: 0.9899
Test accuracy: 0.9898999929428101


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [8]:
model.save('mnist_model.h5')

In [11]:
model_size = os.path.getsize('mnist_model.h5') / 1024
print(f"La taille du modèle est de {model_size:.2f} ko")

La taille du modèle est de 994.52 ko


In [12]:
param_size = model_size / model.count_params()
print(f"La taille moyenne d'un paramètre est de {param_size:.4f} ko")

La taille moyenne d'un paramètre est de 0.0082 ko


In [15]:
# Charger le modèle Keras
model = tf.keras.models.load_model('mnist_model.h5')

# Convertir le modèle Keras en modèle TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [16]:
# Convertir le modèle TensorFlow Lite
tflite_model = converter.convert()

# Sauvegarder le modèle TensorFlow Lite dans un fichier
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\haddr\AppData\Local\Temp\tmprdai5g_g\assets


INFO:tensorflow:Assets written to: C:\Users\haddr\AppData\Local\Temp\tmprdai5g_g\assets


In [17]:
# Calculer la taille du fichier TensorFlow Lite
tflite_size = os.path.getsize('model.tflite') / 1024
print(f"La taille du modèle TensorFlow Lite est de {tflite_size:.2f} ko")

# Calculer le rapport entre la taille du modèle TensorFlow Lite et la taille du modèle Keras
compression_ratio = tflite_size / model_size
print(f"Le modèle TensorFlow Lite est {compression_ratio:.2f} fois plus petit que le modèle Keras")

La taille du modèle TensorFlow Lite est de 479.61 ko
Le modèle TensorFlow Lite est 0.48 fois plus petit que le modèle Keras


In [18]:
# Recréer le convertisseur
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Ajouter l'optimisation par défaut à la liste d'optimisations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [19]:
# Convertir le modèle TensorFlow Lite avec les optimisations spécifiées
tflite_model = converter.convert()

# Sauvegarder le modèle TensorFlow Lite dans un fichier
with open('model_optimized.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\haddr\AppData\Local\Temp\tmpec7vdd0j\assets


INFO:tensorflow:Assets written to: C:\Users\haddr\AppData\Local\Temp\tmpec7vdd0j\assets


In [20]:
# Calculer la taille du fichier TensorFlow Lite optimisé
tflite_optimized_size = os.path.getsize('model_optimized.tflite') / 1024
print(f"La taille du modèle TensorFlow Lite optimisé est de {tflite_optimized_size:.2f} ko")

# Calculer le rapport entre la taille du modèle TensorFlow Lite optimisé et la taille du modèle Keras
compression_ratio = tflite_optimized_size / model_size
print(f"Le modèle TensorFlow Lite optimisé est {compression_ratio:.2f} fois plus petit que le modèle Keras")

La taille du modèle TensorFlow Lite optimisé est de 126.45 ko
Le modèle TensorFlow Lite optimisé est 0.13 fois plus petit que le modèle Keras
